## Calculate Dotcom Order Ratio

In [1]:
import numpy as np
import pandas as pd
import pyodbc
conn = pyodbc.connect('DSN=teradata_odbc')

In [2]:
qry = '''
SELECT * FROM PRD_FOL_TMP.OCP_SUPERCAT_YX WHERE CATEGORY LIKE '%INK%'
'''
df = pd.read_sql(sql = qry,con = conn)
df.head()

,SITE_ID,SEG_ID,CORP_TXN_ID,VISIT_TYPE_CODE,CATEGORY
0,01058181430017891466,5,9793376735,D,1_InkandToner
1,00659016580273276800,5,9793257471,D,1_InkandToner
2,00159929590011709537,5,2331082445,R,1_InkandToner
3,02831381850020064586,5,9791716338,D,1_InkandToner
4,01543951640239698933,5,9794898839,D,1_InkandToner


In [4]:
# calculate number of online and retail transactions for each site 
df.groupby(['SITE_ID','VISIT_TYPE_CODE'])['CORP_TXN_ID'].nunique().head()

SITE_ID               VISIT_TYPE_CODE
00000000010000017409  D                  3
                      R                  1
00000000010000120529  D                  1
00000000010000170960  D                  1
                      R                  2
Name: CORP_TXN_ID, dtype: int64

In [6]:
# pivot table
df_pivot = df.pivot_table(index = 'SITE_ID', columns = 'VISIT_TYPE_CODE',values = 'CORP_TXN_ID', aggfunc = len,margins = True,fill_value = 0)
df_pivot.head()

VISIT_TYPE_CODE,D,R,All
SITE_ID,,,
00000000010000017409,3,1,4
00000000010000120529,1,0,1
00000000010000170960,1,2,3
00000000010000187540,0,2,2
00000000010000547718,0,16,16


In [7]:
type(df_pivot)

pandas.core.frame.DataFrame

In [4]:
# add column to calculate online/corp order ratio for each site
# iloc function to access element or column in pivot table
df_pivot['ratio'] = df_pivot.iloc[:,0]/df_pivot.iloc[:,2]
df_pivot.head()

VISIT_TYPE_CODE,D,R,All,ratio
SITE_ID,,,,
00000000010000017409,3,1,4,0.750000
00000000010000120529,1,0,1,1.000000
00000000010000170960,1,2,3,0.333333
00000000010000187540,0,2,2,0.000000
00000000010000547718,0,16,16,0.000000


In [7]:
pivo_des = df_pivot['ratio'].describe()
# online order ratio mean is 0.65 for ink & toner category
pivo_des

count    278845.000000
mean          0.652750
std           0.372418
min           0.000000
25%           0.400000
50%           0.750000
75%           1.000000
max           1.000000
Name: ratio, dtype: float64

In [9]:
pivo_des['50%']

0.75

In [10]:
df_pivot.head().to_excel(r'C:\Users\Xiayu001\Desktop\export_dataframe.xlsx')

In [10]:
import sqlalchemy
from sqlalchemy import create_engine
import sqlalchemy_teradata
import pandas as pd

# connection info
user = 'MKTG_PRDFOCOPLETL'
pw = 'staples123'
host = 'td-mrk-p-'
td_engine = create_engine('teradata://'+ user +':' + pw + '@'+ host + ':22/')

# write your DataFrame df to a table called df_test in the sql database
df.head().to_sql("df_test_1", td_engine, if_exists = 'replace',schema='PRD_FOL_TMP',index = False)